In [1]:
import pandas as pd
import numpy as np
from keras.models import load_model
import cornac.models.recommender as cmr
import math

Using TensorFlow backend.


In [2]:
def mag(x): return math.sqrt(sum(i**2 for i in x))

In [3]:
def cosine_similarity(p_m, u_m, user_id):
    cos_sim = []
    
    u_vec = u_m.loc[user_id,:]
    u_size = mag(u_vec)

    for i in range(len(p_m)):
        p_vec = p_m.iloc[i,:]
        p_size = mag(p_vec)
        cos_sim.append(np.dot(p_vec, u_vec)/(p_size*u_size))
        
    return cos_sim

In [72]:
#Eat_contents_based
def Eat_recommendation(u_id):
     # 0. preprocess place_matrix
    place_matrix = pd.read_csv('../../Eat/data/normalized_item_matrix.csv')
    cbf = place_matrix[['p_id']]
    place_matrix.set_index('p_id', inplace=True)
    
     # 1. preprocess user_matrix
    user_matrix = pd.read_csv('../../Eat/data/normalized_user_matrix.csv')
    user_id  = 'u1s2&'+u_id
    user_index = user_matrix[user_matrix['u_id']==user_id].index[0]
    del user_matrix['u_id']
    
    # 2. cbf result
    cbf_result = cosine_similarity(place_matrix, user_matrix, user_index)
    cbf['cbf'] = cbf_result
    cbf.set_index('p_id', inplace=True, drop=True)
    print('Complete to get Eat - contents_based result')
    
    # 3. ncf result
    ncf = pd.read_csv('../../Eat/data/predictions/ncf_scores.csv')
    ncf.rename(columns = {'ncf_score':'ncf'}, inplace=True)
    ncf = ncf[ncf['u_id'] == u_id]
    del ncf['u_id']
    ncf['p_id'] = ncf['p_id'].astype(int)
    ncf.set_index('p_id', inplace=True, drop=True)
    print('Complete to get Eat - ncf result')
    
    # 4. make_matrix
    eat_info =  pd.read_csv('../../Eat/data/translated_eat_info.csv')
    eat_info = pd.DataFrame(eat_info, columns=['p_id','name','address'])
    eat_info.set_index('p_id', inplace=True, drop=True)
    eat_info['district'] = 0
    for i in range (0, len(eat_info['address'])):
        eat_info['district'][i] = eat_info['address'][i].split(' ')[1]
    print('Complete to make Eat -Result matrix')
    
    # 5. join more
    result = eat_info.join(cbf, how='left').join(ncf, how='left')
    result['result'] = result['cbf'] * result['ncf']
    result.sort_values(by=['result'], inplace=True, ascending=False)
    
    return result;

In [73]:
Eat_recommendation('쥬')

/Users/ryeonju/miniconda3/envs/reco_base/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3254: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/Users/ryeonju/miniconda3/envs/reco_base/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.
/Users/ryeonju/miniconda3/envs/reco_base/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


Complete to get Eat - contents_based result
Complete to get Eat - ncf result


/Users/ryeonju/miniconda3/envs/reco_base/lib/python3.6/site-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Complete to make Eat -Result matrix


,name,address,district,cbf,ncf,result
p_id,,,,,,
972,더훈,서울특별시 용산구 독서당로 87,용산구,0.117719,0.999707,0.117684
2778,스시올로지,서울특별시 마포구 동교로 266-11,마포구,0.134336,0.431962,0.058028
5186,할리스커피,서울특별시 금천구 가산동 459-11,금천구,0.303065,0.035623,0.010796
5281,호 파스타,서울시 광진구 화양동 12-52,광진구,0.072217,0.103250,0.007456
2892,식샤,서울특별시 성북구 성북동1가 36-1,성북구,0.064116,0.071780,0.004602
...,...,...,...,...,...,...
5415,후니도니,서울특별시 종로구 종로 19 르메이에르종로타운 B1,종로구,0.115497,NaN,NaN
1519,맛짱,서울특별시 중랑구 중랑역로 116,중랑구,NaN,0.000036,NaN
3843,이탈리아노501,서울특별시 도봉구 해등로32길 76,도봉구,NaN,0.000000,NaN


In [74]:
#Go_contents_based
def Go_recommendation(u_id):
    # 0. preprocess place_matrix
    place_matrix = pd.read_csv('../../Go/data/normalized_item_matrix.csv')
    cbf = place_matrix[['p_id']]
    del place_matrix['total_count']
    place_matrix.set_index('p_id', inplace=True)
    
    # 1. preprocess user_matrix
    user_matrix = pd.read_csv('../../Go/data/normalized_user_matrix.csv')
    user_id  = 'u1s2&'+u_id
    user_index = user_matrix[user_matrix['u_id']==user_id].index[0]
    del user_matrix['u_id']
    del user_matrix['total_count']
    
    # 2. cbf result
    cbf_result = cosine_similarity(place_matrix, user_matrix, user_index)
    cbf['cbf'] = cbf_result
    cbf.set_index('p_id', inplace=True, drop=True)
    print('Complete to get Go - contents_based result')
    
    # 3. ncf result
    ncf = pd.read_csv('../../Go/data/ncf_scores2.csv')
    ncf.rename(columns = {'ncf_score':'ncf'}, inplace=True)
    ncf = ncf[ncf['u_id'] == u_id]
    del ncf['u_id']
    ncf.set_index('p_id', inplace=True, drop=True)
    print('Complete to get Go - ncf result')
    
    # 4. make_matrix
    go_info =  pd.read_csv('../../Go/data/item_info.csv')
    go_info = pd.DataFrame(go_info, columns=['p_id','p_name','address'])
    go_info['district'] = 0;
    for i in range (0, len(go_info['address'])):
        try:
            if ' ' in go_info['address'][i]:
                go_info['district'][i] = go_info['address'][i].split(' ')[1]
            else:
                go_info['district'][i] = go_info['address'][i]
        except:
            pass
    go_info.set_index('p_id', inplace=True, drop=True)
    print('Complete to make Go - Result matrix')
    
    # 5. join more
    result = go_info.join(cbf, how='left').join(ncf, how='left')
    result['result'] = result['cbf'] * result['ncf']
    result.dropna(inplace=True)
    result.sort_values(by=['result'], inplace=True, ascending=False)
    
    return result;

In [76]:
Go_recommendation('쥬')

/Users/ryeonju/miniconda3/envs/reco_base/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.
/Users/ryeonju/miniconda3/envs/reco_base/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Complete to get Go - contents_based result
Complete to get Go - ncf result


/Users/ryeonju/miniconda3/envs/reco_base/lib/python3.6/site-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/ryeonju/miniconda3/envs/reco_base/lib/python3.6/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


Complete to make Go - Result matrix


,p_name,address,district,cbf,ncf,result
p_id,,,,,,
g84,도깨비코인노래연습장,서울 광진구 자양로18길 19 해민빌딩 3층 (우)05043,광진구,0.171600,0.263659,0.045244
f231,신촌반지클럽,서울 서대문구 연세로 25-1 5층 (우)03788,서대문구,0.244371,0.075374,0.018419
3,국립민속박물관&국립민속박물관 어린이박물관,종로구,종로구,0.222064,0.067447,0.014978
c113,해브어코믹스데이,서울 마포구 도화4길 23 2층 (우)04169,마포구,0.159505,0.028977,0.004622
a59,스핀볼링센터 가든5점,서울 송파구 충민로 52 지하1층 57호 (우)05839,송파구,0.186089,0.023632,0.004398
...,...,...,...,...,...,...
387,장충동 족발 골목,중구,중구,0.000000,0.000010,0.000000
e38,엑스케이프 홍대점,서울 마포구 홍익로3길 44 호곡빌딩 7층 (우)04039,마포구,0.000000,0.000106,0.000000
e63,셜록홈즈 영등포점,서울 영등포구 영중로4길 25-1 4층 (우)07304,영등포구,0.000000,0.000014,0.000000


In [41]:
#Watch_contents_based
def Watch_recommendation(u_id):
    # 0. preprocess place_matrix
    place_matrix = pd.read_csv('../../Watch/data/normalized_item_matrix.csv')
    cbf = place_matrix[['p_id']]
    place_matrix.set_index('p_id', inplace=True)
    
     # 1. preprocess user_matrix
    user_matrix = pd.read_csv('../../Watch/data/normalized_user_matrix.csv')
    user_id  = 'u1s2&'+u_id
    user_index = user_matrix[user_matrix['u_id']==user_id].index[0]
    del user_matrix['u_id']
    
    # 2. cbf result
    cbf_result = cosine_similarity(place_matrix, user_matrix, user_index)
    cbf['cbf'] = cbf_result
    cbf.set_index('p_id', inplace=True, drop=True)
    print('Complete to get Watch - contents_based result')
    
    # 3. ncf result
    ncf = pd.read_csv('../../Watch/data/predictions/ncf_scores.csv')
    ncf.rename(columns = {'ncf_score':'ncf'}, inplace=True)
    ncf = ncf[ncf['u_id'] == u_id]
    del ncf['u_id']
    ncf['p_id'] = ncf['p_id'].astype(int)
    ncf.set_index('p_id', inplace=True, drop=True)
    print('Complete to get Watch - ncf result')
    
    # 4. make_matrix
    watch_info =  pd.read_csv('../../Watch/data/current_movie.csv')
    watch_info = pd.DataFrame(watch_info, columns=['id','name'])
    watch_info.set_index('id', inplace=True, drop=True)
    print('Complete to make Watch Result matrix')
    
    # 5. join cbf,ncf
    result = watch_info.join(cbf, how='left').join(ncf, how='left')
    result['result'] = result['cbf'] * result['ncf']
    
    # 6.get movie_name
    result.sort_values(by=['result'], ascending=False, inplace=True)
    reco_movie_name = result.iloc[0]['name']
    
    return reco_movie_name;

In [42]:
def get_preferred_district(u_id):
    user_survey_df = pd.read_csv('../data/survey_result.csv')
    district = user_survey_df[user_survey_df['u_id'] == u_id]['District'].values[0]
    return district.split(', ')

In [98]:
def Search_Recommendable_Places(eat_matrix, go_matrix, u_id):
    dist_ls = get_preferred_district(u_id)
    eat_matrix = eat_matrix[eat_matrix['district'].isin(dist_ls)]
    go_matrix = go_matrix[go_matrix['district'].isin(dist_ls)]
    
    eat_matrix.reset_index(inplace=True, drop=True)
    go_matrix.reset_index(inplace=True, drop=True)
    
    trial=0
    e_i=0
    g_i=0
    
    if len(dist_ls) > 1 :
        if (eat_matrix['district'][0] == go_matrix['district'][0]):
            print('베스트 행정구 일치!')
            eat_res =  eat_matrix.iloc[0]
            go_res = go_matrix.iloc[0]
            return (eat_res, go_res)
        
        else:
            while (eat_matrix['district'][0] != go_matrix['district'][g_i]):
                g_i = g_i + 1
            eat_res1 =  eat_matrix.iloc[0]
            go_res1 = go_matrix.iloc[g_i]
            
            while (eat_matrix['district'][e_i] != go_matrix['district'][0]):
                e_i = e_i + 1
            eat_res2 =  eat_matrix.iloc[e_i]
            go_res2 = go_matrix.iloc[0]
            
            return(eat_res1, go_res1, eat_res2, go_res2)
        
#         while (e_i< len(eat_matrix) and g_i<len(go_matrix)):
#             if (eat_matrix['district'][e_i] == go_matrix['district'][g_i]):
#                 eat_res =  eat_matrix.iloc[e_i]
#                 go_res = go_matrix.iloc[g_i]
#                 return (eat_res, go_res)
                
#             if (trial%2 == 0):
#                 e_i = e_i+1
#             else:
#                 g_i = g_i+1
#             trial = trial + 1
    else:
        eat_res = eat_matrix[eat_matrix['district']==dist_ls[0]].iloc[0]
        go_res = go_matrix[go_matrix['district']==dist_ls[0]].iloc[0]
        return (eat_res, go_res)

In [99]:
def save_each_user_reco_item(user_id, eat, go, watch):
    pd.to_csv(f'../data/')

In [128]:
def main():
    u_id = input('Enter User ID: ')
    go = Go_recommendation(u_id)
    eat = Eat_recommendation(u_id)
    watch = Watch_recommendation(u_id)
    result = Search_Recommendable_Places(eat, go, u_id)
    if len(result) == 2:
        eat_result = result[0]
        go_result = result[1]
        print(f'{u_id}님께 추천드릴 맛집은 {eat_result["district"]}에 위치한 {eat_result["name"]}입니다.')
        print(f'{u_id}님께 추천드릴 데이트 장소는 {go_result["district"]}에 위치한 {go_result["p_name"]}입니다.')
    elif len(result) == 4:
        eat_result1 = result[0]
        go_result1 = result[1]
        eat_result2 = result[2]
        go_result2 = result[3]
        print(f'{u_id}님! {eat_result1["district"]}에서의 데이트를 계획하고 계신가요?')
        print(f'{u_id}님께 추천드릴 맛집은 {eat_result1["district"]}에 위치한 {eat_result1["name"]}입니다.')
        print(f'{u_id}님께 추천드릴 데이트 장소는 {go_result1["district"]}에 위치한 {go_result1["p_name"]}입니다.')
        print(f'혹시나! {eat_result2["district"]}에서의 데이트를 계획하고 계신다면?')
        print(f'맛집으로는 {eat_result2["district"]}에 위치한 {eat_result2["name"]}와 ')
        print(f'데이트 장소로는 {go_result2["district"]}에 위치한 {go_result2["p_name"]}을(를) 추천드려요.')
        
    print(f'영화가 끌리는 데이트라면 현재 상영중인 영화 {watch}은(는) 어떠신가요?')

In [129]:
main()

Enter User ID: hhhhhhhhhh


/Users/ryeonju/miniconda3/envs/reco_base/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.
/Users/ryeonju/miniconda3/envs/reco_base/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/ryeonju/miniconda3/envs/reco_base/lib/python3.6/site-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Complete to get Go - contents_based result
Complete to get Go - ncf result
Complete to make Go - Result matrix


/Users/ryeonju/miniconda3/envs/reco_base/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


Complete to get Eat - contents_based result
Complete to get Eat - ncf result


/Users/ryeonju/miniconda3/envs/reco_base/lib/python3.6/site-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Complete to make Eat -Result matrix
Complete to get Watch - contents_based result
Complete to get Watch - ncf result
Complete to make Watch Result matrix
hhhhhhhhhh님! 강남구에서의 데이트를 계획하고 계신가요?
hhhhhhhhhh님께 추천드릴 맛집은 강남구에 위치한 쿠이신보입니다.
hhhhhhhhhh님께 추천드릴 데이트 장소는 강남구에 위치한 라스트이스케이프입니다.
혹시나! 서대문구에서의 데이트를 계획하고 계신다면?
맛집으로는 서대문구에 위치한 그릭데이와 
데이트 장소로는 서대문구에 위치한 신촌반지클럽을(를) 추천드려요.
영화가 끌리는 데이트라면 현재 상영중인 영화 어느 가족은(는) 어떠신가요?


In [80]:
def a():
    return 1, 2

In [84]:
b = a()

In [85]:
b

(1, 2)

In [88]:
f'${b}'

'$(1, 2)'